The program is used to simulate the process -- consumers sending their orders to the salers online, and salers dispatch the goods after packing up them.

People buy different amount of goods every day, and the number of people sending their orders varies each day.

In [238]:
import simpy
import random

In [239]:
#The time period of the simulation
days=30

#The number of people who send their orders in each day
people_num=[]
for i in range(days):
    people_num.append(random.randint(1,10))

#There are 2 kinds of goods to choose(goods A and goods B)
#The number of goods bought by each people from each day
orderA={}
orderB={}
finished_consumer=[]
total_orders=0

for i in range(days):
    a=[]
    b=[]
    for j in range(people_num[i]):
        a.append(random.randint(1,10))
        b.append(random.randint(1,5))
        total_orders=total_orders+1    
    orderA[i]=a
    orderB[i]=b

#containers
    #goods
goodsA_capacity = 100000
initial_goodsA = 100
goodsB_capacity=100000
initial_goodsB=100
    
    #cardboards(cardboards are used to package the goods)
cardboards_capacity = 100000
initial_cardboards = 200

    #orders made
orderA_capacity = 10000
orderB_capacity = 10000

    #goods that have been packaged
packagedA_capacity = 10000
packagedB_capacity = 10000

    #goods that have been dispatched
dispatchedA_capacity=10000
dispatchedB_capacity=10000

    #goods that have been received successfully by consumers
receivedA_capacity=10000
receivedB_capacity=10000

#critical levels
    #The least amount of goodsA, goods B and cardboards
goodsA_critial_stock = 10*10*2 #1 days to deliver + 1 marging, maximum 10 people buy 10 goodsA every time
goodsB_critial_stock = 10*5*3 #2 days to deliver + 1 marging, maximum 10 people buy 5 goodsB every time
cardboards_critical_stock = ((10*10)+(10*5))*2 #1 day to deliver + 1 marging


In [240]:
class Logistics:
    def __init__(self, env):
        self.goodsA = simpy.Container(env, capacity = goodsA_capacity, init = initial_goodsA)
        self.goodsA_control = env.process(self.goodsA_amount_control(env))
        
        self.goodsB = simpy.Container(env, capacity = goodsB_capacity, init = initial_goodsB)
        self.goodsB_control = env.process(self.goodsB_amount_control(env))
        
        self.cardboards = simpy.Container(env, capacity = cardboards_capacity, init = initial_cardboards)
        self.cardboards_control = env.process(self.cardboards_stock_control(env))
        
        self.orderA = simpy.Container(env, capacity = orderA_capacity, init = 0)
        self.orderB = simpy.Container(env, capacity = orderB_capacity, init = 0)
        
        self.packagedA = simpy.Container(env ,capacity = packagedA_capacity, init = 0)
        self.packagedB = simpy.Container(env ,capacity = packagedB_capacity, init = 0)

        self.dispatchedA = simpy.Container(env ,capacity = dispatchedA_capacity, init = 0)
        self.dispatchedB = simpy.Container(env ,capacity = dispatchedB_capacity, init = 0)

        self.receivedA = simpy.Container(env ,capacity = receivedA_capacity, init = 0)
        self.receivedB = simpy.Container(env ,capacity = receivedB_capacity, init = 0)

        self.orders_count = simpy.Container(env ,capacity = 50000, init = 0) #caculate how many orders have been successfully finished


 #goods A, goods B and cardboards will be supplied when their amount is lower than the critial level   
    def goodsA_amount_control(self,env):
        yield env.timeout(0)
        while True:
            if self.goodsA.level <= goodsA_critial_stock:
                print('Goods A stock bellow critical level (now is {0}) at day {1}'.format(
                    self.goodsA.level, int(env.now)+1))
                print('Need more Goods A')
                print('----------------------------------')
                yield env.timeout(2)#need 2 days to supply
                print('Goods A arrived at day {0}'.format(
                    int(env.now)))
                print('----------------------------------')
                n=10*10*2
                yield self.goodsA.put(n)
                print('The amount of Goods A is {0} now'.format(
                    self.goodsA.level))
                print('----------------------------------')
                yield env.timeout(1)
            else:
                yield env.timeout(1)
    
    
    def goodsB_amount_control(self,env):
        yield env.timeout(0)
        while True:
            if self.goodsB.level <= goodsB_critial_stock:
                print('Goods B stock bellow critical level (now is {0}) at day {1}'.format(
                    self.goodsB.level, int(env.now)+1))
                print('Need more Goods B')
                print('----------------------------------')
                yield env.timeout(3)#need 3 days to supply
                print('Goods B arrived at day {0}'.format(
                    int(env.now)))
                print('----------------------------------')
                n=10*5*3
                yield self.goodsB.put(n)
                print('The amount of Goods B is {0} now'.format(
                    self.goodsB.level))
                print('----------------------------------')
                yield env.timeout(1)
            else:
                yield env.timeout(1)
            

    def cardboards_stock_control(self,env):
        yield env.timeout(0)
        while True:
            if self.cardboards.level <= cardboards_critical_stock:
                print('Cardboards stock bellow critical level (now is {0}) at day {1}'.format(
                    self.cardboards.level, int(env.now)+1))
                print('Need more cardboards to make boxes')
                print('----------------------------------')
                yield env.timeout(1)#need 1 day to supply
                print('Cardboards arrived at day {0}'.format(
                    int(env.now)))
                print('----------------------------------')
                n=10*15*2
                yield self.cardboards.put(n)
                print('The amount of cardboards is {0} now'.format(
                    self.cardboards.level))
                print('----------------------------------')
                yield env.timeout(1)
            else:
                yield env.timeout(1)

In [241]:
#Logistics.orderA will store the orders made on goods A every day, and the same is for goods B.    
def consumer_buy(env, Logistics):
    while True:
        for i in range(days):
            print('Day {0}'.format(
                int(env.now)+1))
            print('------')
            for j in range(people_num[i]):
                n=j+1
                a=orderA[i]
                b=orderB[i]
                yield Logistics.orderA.put(a[j])
                yield Logistics.orderB.put(b[j])
                print('Costomer {0}-{1} ordered {2} goods A and {3} goods B at day {4}'.format(
                i+1,n,a[j], b[j],int(env.now)+1))#Costomers are distinguished by day and number
            print('----------------------------------')
            yield env.timeout(1)
   
#Goods A need to be packaged after being dispatched and it takes 1/48 day(30min) for each. 
def salerA(env, Logistics):
    while True:
        for i in range(days):
            for j in range(people_num[i]):
                a=orderA[i]
                yield Logistics.orderA.get(a[j])
                yield Logistics.goodsA.get(a[j])
                yield Logistics.cardboards.get(a[j])
                yield Logistics.packagedA.put(a[j])
            packageA_time = 1/48
            yield env.timeout(packageA_time)

#Goods B need to be packaged after being dispatched and it takes 1/24 day(1h) for each.                
def salerB(env, Logistics):
    while True:
        for i in range(days):
            for j in range(people_num[i]):
                b=orderB[i]
                yield Logistics.orderB.get(b[j])
                yield Logistics.goodsB.get(b[j])
                yield Logistics.cardboards.get(b[j])
                yield Logistics.packagedB.put(b[j])
            packageB_time = 1/24
            yield env.timeout(packageB_time)
        
#After goods are being packaged, orders will be dispatched to each consumer by couriers. It will take 1~2 days.
def courier(env, Logistics):
    while True:
        for i in range(days):
            for j in range(people_num[i]):
                n=j+1
                a=orderA[i]
                b=orderB[i]
                yield Logistics.packagedA.get(a[j])
                yield Logistics.packagedB.get(b[j])
                dispatch_time=random.randint(1,2)
                yield env.timeout(dispatch_time/people_num[i])#Qurantee that orders of the same-day costomers can be dispatched in the same day.
                yield Logistics.dispatchedA.put(a[j])
                yield Logistics.dispatchedB.put(b[j])
                print('{0} Goods A and {1} Goods B are dispatched for costomer {2}-{3} at day {4}'.format(
                a[j], b[j],i+1,n,int(env.now)+1))
                print('----------------------------------')

#After consumers receiving their goods sucessfully, Logistics.receivedA & B will store how many goods have been sold in total.
def consumer_receive(env, Logistics):
    while True:
        for i in range(days):
            for j in range(people_num[i]):
                n=j+1
                a=orderA[i]
                b=orderB[i]
                yield Logistics.dispatchedA.get(a[j])
                yield Logistics.dispatchedB.get(b[j])
                yield env.timeout(0)
                yield Logistics.receivedA.put(a[j])
                yield Logistics.receivedB.put(b[j])
                yield Logistics.orders_count.put(1)
                finished_consumer.append('Person '+str(i+1)+'-'+str(n))
                print('Costomer {0}-{1} order is finished: {2} Goods A and {3} Goods B are received at day {4}'.format(
                i+1,n,a[j], b[j],int(env.now)+1))
                print('----------------------------------')
            

In [242]:
print(f'STARTING SIMULATION')
print(f'----------------------------------')
print(f'----------------------------------')

#Simulation start
env = simpy.Environment()
logistics = Logistics(env)


consumerbuy_gen = env.process(consumer_buy(env, logistics))
saler_A_gen = env.process(salerA(env, logistics))
saler_B_gen = env.process(salerB(env, logistics))
Courier_gen = env.process(courier(env, logistics))
consumerreceive_gen = env.process(consumer_receive(env, logistics))

env.run(until = days)

STARTING SIMULATION
----------------------------------
----------------------------------
Day 1
------
Goods A stock bellow critical level (now is 100) at day 1
Need more Goods A
----------------------------------
Goods B stock bellow critical level (now is 100) at day 1
Need more Goods B
----------------------------------
Cardboards stock bellow critical level (now is 200) at day 1
Need more cardboards to make boxes
----------------------------------
Costomer 1-1 ordered 9 goods A and 2 goods B at day 1
Costomer 1-2 ordered 7 goods A and 4 goods B at day 1
Costomer 1-3 ordered 3 goods A and 4 goods B at day 1
Costomer 1-4 ordered 10 goods A and 5 goods B at day 1
Costomer 1-5 ordered 1 goods A and 1 goods B at day 1
Costomer 1-6 ordered 10 goods A and 3 goods B at day 1
Costomer 1-7 ordered 2 goods A and 3 goods B at day 1
Costomer 1-8 ordered 8 goods A and 1 goods B at day 1
----------------------------------
9 Goods A and 2 Goods B are dispatched for costomer 1-1 at day 1
----------

In [243]:
print('There are {0} Goods A and {1} Goods B left'.format(
    logistics.goodsA.level, logistics.goodsB.level))
print('There are {0} Goods A and {1} Goods B packaged'.format(
    logistics.packagedA.level, logistics.packagedB.level))
print(f'----------------------------------')

print('Total orders are: {0}'.format(total_orders))
print('Total orders finished: {0} '.format(logistics.orders_count.level))
print('Finished rate is: {0}%'.format(format(logistics.orders_count.level/total_orders*100,'.2f')))
print('Finished costomers are:')
print(finished_consumer)
print('{0} Goods A and {1} Goods B have been sent to the costomers successfully'.format(logistics.receivedA.level,logistics.receivedB.level))
print(f'----------------------------------')
print(f'----------------------------------')
print(f'SIMULATION COMPLETED')

There are 156 Goods A and 142 Goods B left
There are 139 Goods A and 91 Goods B packaged
----------------------------------
Total orders are: 181
Total orders finished: 148 
Finished rate is: 81.77%
Finished costomers are:
['Person 1-1', 'Person 1-2', 'Person 1-3', 'Person 1-4', 'Person 1-5', 'Person 1-6', 'Person 1-7', 'Person 1-8', 'Person 2-1', 'Person 2-2', 'Person 2-3', 'Person 2-4', 'Person 2-5', 'Person 2-6', 'Person 3-1', 'Person 3-2', 'Person 4-1', 'Person 4-2', 'Person 4-3', 'Person 4-4', 'Person 4-5', 'Person 4-6', 'Person 4-7', 'Person 4-8', 'Person 5-1', 'Person 5-2', 'Person 5-3', 'Person 5-4', 'Person 5-5', 'Person 5-6', 'Person 6-1', 'Person 6-2', 'Person 6-3', 'Person 6-4', 'Person 6-5', 'Person 6-6', 'Person 6-7', 'Person 6-8', 'Person 6-9', 'Person 6-10', 'Person 7-1', 'Person 7-2', 'Person 7-3', 'Person 7-4', 'Person 7-5', 'Person 8-1', 'Person 8-2', 'Person 8-3', 'Person 8-4', 'Person 8-5', 'Person 8-6', 'Person 8-7', 'Person 9-1', 'Person 9-2', 'Person 9-3', 'Pers

We get the whole logistics information in a certain number of days, and we can see how many goods are ready in the saler's factory and how many goods have been sold out successfully. The result of finished rate is good. We can also see how many orders have been finished and who have received their goods.